# Summarization of Financial Data Using a Large Language Model (LLM)

Document a large language model using the ValidMind Developer Framework. The use case is a summarization of financial news (https://huggingface.co/datasets/cnn_dailymail).

This interactive notebook shows you how to:

- Initialize the ValidMind Developer Framework
- Run various tests to quickly generate documentation about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now!**](https://app.prod.validmind.ai)
:::

This notebook requires an OpenAI API secret key to run. If you don't have one, visit [API keys](https://platform.openai.com/account/api-keys) on OpenAI's site to create a new key for yourself. Note that API usage charges may apply.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **LLM-based Text Summarization** as the template and **Marketing/Sales - Analytics** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
# Replace with your code snippet
import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

## Load data

### CNN dataset

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [ ]:
# lets first install the `datasets` library from huggingface
%pip install -q datasets

In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')

train_df = cnn_dataset.data['train'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()

train_df = train_df[['article', 'highlights']]
test_df = test_df[['article', 'highlights']]

train_df = train_df.head(5)
test_df = test_df.head(5)

## Model selection

In [ ]:
from validmind.models import FoundationModel, Prompt
import os
import dotenv
import openai

dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")


def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

## Prompt engineering

In [ ]:
prompt_template = """
You are an AI with expertise in summarizing financial news.
Your task is to provide a concise summary of the specific news article provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the article.

Article to Summarize:

```
{article}
```

Please respond with a concise summary of the article's main points.
Ensure that your summary is based on the content of the article and not on external information or assumptions.
""".strip()

prompt_variables = ["article"]

## Initialize ValidMind datasets and models

In [ ]:
vm_test_ds = vm.init_dataset(
    dataset=test_df,
    text_column="article",
    target_column="highlights",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

## Model performance metrics

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RougeMetricsAggregate",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.BertScoreAggregate",
    model=vm_model)

## Bias metrics

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityHistogram",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.ToxicityScore",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardScore",
    model=vm_model)

In [ ]:
metric = vm.tests.run_test(
    "validmind.model_validation.RegardHistogram",
    model=vm_model)